# HLL: Manaul Conversion (from Cirq to Qiskit)

Source: https://kth.instructure.com/courses/30263/pages/cirq-codes-qft-grover-and-hll?module_item_id=392478

## Object/API Level Conversion

In [29]:
"""
Demonstrates the algorithm for solving linear systems by Harrow, Hassidim,
Lloyd (HHL).
The HHL algorithm solves a system of linear equations, specifically equations
of the form Ax = b, where A is a Hermitian matrix, b is a known vector, and
x is the unknown vector. To solve on a quantum system, b must be rescaled to
have magnitude 1, and the equation becomes:
|x> = A**-1 |b> / || A**-1 |b> ||
The algorithm uses 3 sets of qubits: a single ancilla qubit, a register (to
store eigenvalues of A), and memory qubits (to store |b> and |x>). The
following are performed in order:
1) Quantum phase estimation to extract eigenvalues of A
2) Controlled rotations of ancilla qubit
3) Uncomputation with inverse quantum phase estimation
For details about the algorithm, please refer to papers in the
REFERENCE section below. The following description uses variables defined
in the HHL paper.
This example is an implementation of the HHL algorithm for arbitrary 2x2
Hermitian matrices. The output of the algorithm are the expectation values
of Pauli observables of |x>. Note that the accuracy of the result depends
on the following factors:
* Register size
* Choice of parameters C and t
The result is perfect if
* Each eigenvalue of the matrix is in the form
  2π/t * k/N,
  where 0≤k<N, and N=2^n, where n is the register size. In other words, k is a
  value that can be represented exactly by the register.
* C ≤ 2π/t * 1/N, the smallest eigenvalue that can be stored in the circuit.
The result is good if the register size is large enough such that for every
pair of eigenvalues, the ratio can be approximated by a pair of possible
register values. Let s be the scaling factor from possible register values to
eigenvalues. One way to set t is
t = 2π/sN
For arbitrary matrices, because properties of their eigenvalues are typically
unknown, parameters C and t are fine-tuned based on their condition number.
=== REFERENCE ===
Harrow, Aram W. et al. Quantum algorithm for solving linear systems of
equations (the HHL paper)
https://arxiv.org/abs/0811.3171
Coles, Eidenbenz et al. Quantum Algorithm Implementations for Beginners
https://arxiv.org/abs/1804.03719
=== CIRCUIT ===
Example of circuit with 2 register qubits.
(0, 0): ─────────────────────────Ry(θ )─Ry(θ )─Ry(θ )─Ry(θ )──────────────M──₄₁₂₃
                     ┌──────┐    │      │      │      │ ┌───┐
(1, 0): ─H─@─────────│      │──X─@──────@────X─@──────@─│   │─────────@─H────
           │         │QFT^-1│    │      │      │      │ │QFT│         │
(2, 0): ─H─┼─────@───│      │──X─@────X─@────X─@────X─@─│   │─@───────┼─H────
           │     │   └──────┘                           └───┘ │       │
(3, 0): ───e^iAt─e^2iAt───────────────────────────────────────e^-2iAt─e^-iAt─
Note: QFT in the above diagram omits swaps, which are included implicitly by
reversing qubit order for phase kickbacks.
"""
import math
import numpy as np
import sympy
import cirq

In [30]:
class PhaseEstimation(cirq.Gate):
    """
    A gate for Quantum Phase Estimation.
    unitary is the unitary gate whose phases will be estimated.
    The last qubit stores the eigenvector; all other qubits store the
    estimated phase, in big-endian.
    """
    def __init__(self, num_qubits, unitary):
        self._num_qubits = num_qubits
        self.U = unitary
    def num_qubits(self):
        return self._num_qubits
    def _decompose_(self, qubits):
        qubits = list(qubits)
        yield cirq.H.on_each(*qubits[:-1])
        yield PhaseKickback(self.num_qubits(), self.U)(*qubits)
        yield cirq.qft(*qubits[:-1], without_reverse=True)**-1
class HamiltonianSimulation(cirq.EigenGate, cirq.SingleQubitGate):
    """
    A gate that represents e^iAt.
    This EigenGate + np.linalg.eigh() implementation is used here
    purely for demonstrative purposes. If a large matrix is used,
    the circuit should implement actual Hamiltonian simulation,
    by using the linear operators framework in Cirq for example.
    """
    def __init__(self, A, t, exponent=1.0):
        cirq.SingleQubitGate.__init__(self)
        cirq.EigenGate.__init__(self, exponent=exponent)
        self.A = A
        self.t = t
        ws, vs = np.linalg.eigh(A)
        self.eigen_components = []
        for w, v in zip(ws, vs.T):
            theta = w*t / math.pi
            P = np.outer(v, np.conj(v))
            self.eigen_components.append((theta, P))
    def _with_exponent(self, exponent):
        return HamiltonianSimulation(self.A, self.t, exponent)
    def _eigen_components(self):
        return self.eigen_components
class PhaseKickback(cirq.Gate):
    """
    A gate for the phase kickback stage of Quantum Phase Estimation.
    It consists of a series of controlled e^iAt gates with the memory qubit as
    the target and each register qubit as the control, raised
    to the power of 2 based on the qubit index.
    unitary is the unitary gate whose phases will be estimated.
    """
    def __init__(self, num_qubits, unitary):
        super(PhaseKickback, self)
        self._num_qubits = num_qubits
        self.U = unitary
    def num_qubits(self):
        return self._num_qubits
    def _decompose_(self, qubits):
        qubits = list(qubits)
        memory = qubits.pop()
        for i, qubit in enumerate(qubits):
            yield cirq.ControlledGate(self.U**(2**i))(qubit, memory)
class EigenRotation(cirq.Gate):
    """
    EigenRotation performs the set of rotation on the ancilla qubit equivalent
    to division on the memory register by each eigenvalue of the matrix. The
    last qubit is the ancilla qubit; all remaining qubits are the register,
    assumed to be big-endian.
    It consists of a controlled ancilla qubit rotation for each possible value
    that can be represented by the register. Each rotation is a Ry gate where
    the angle is calculated from the eigenvalue corresponding to the register
    value, up to a normalization factor C.
    """
    def __init__(self, num_qubits, C, t):
        super(EigenRotation, self)
        self._num_qubits = num_qubits
        self.C = C
        self.t = t
        self.N = 2**(num_qubits-1)
    def num_qubits(self):
        return self._num_qubits
    def _decompose_(self, qubits):
        for k in range(self.N):
            kGate = self._ancilla_rotation(k)
            # xor's 1 bits correspond to X gate positions.
            xor = k ^ (k-1)
            for q in qubits[-2::-1]:
                # Place X gates
                if xor % 2 == 1:
                    yield cirq.X(q)
                xor >>= 1
                # Build controlled ancilla rotation
                kGate = cirq.ControlledGate(kGate)
            yield kGate(*qubits)
    def _ancilla_rotation(self, k):
        if k == 0:
            k = self.N
        theta = 2*math.asin(self.C * self.N * self.t / (2*math.pi * k))
        return cirq.ry(theta)
def hhl_circuit(A, C, t, register_size, *input_prep_gates):
    """
    Constructs the HHL circuit.
    Args:
        A: The input Hermitian matrix.
        C: Algorithm parameter, see above.
        t: Algorithm parameter, see above.
        register_size: The size of the eigenvalue register.
        memory_basis: The basis to measure the memory in, one of 'x', 'y', 'z'.
        input_prep_gates: A list of gates to be applied to |0> to generate the
            desired input state |b>.
    Returns:
        The HHL circuit. The ancilla measurement has key 'a' and the memory
        measurement is in key 'm'.  There are two parameters in the circuit,
        `exponent` and `phase_exponent` corresponding to a possible rotation
        applied before the measurement on the memory with a
        `cirq.PhasedXPowGate`.
    """
    ancilla = cirq.LineQubit(0)
    # to store eigenvalues of the matrix
    register = [cirq.LineQubit(i + 1) for i in range(register_size)]
    # to store input and output vectors
    memory = cirq.LineQubit(register_size + 1)
    c = cirq.Circuit()
    hs = HamiltonianSimulation(A, t)
    pe = PhaseEstimation(register_size+1, hs)
    c.append([gate(memory) for gate in input_prep_gates])
    c.append([
        pe(*(register + [memory])),
        EigenRotation(register_size + 1, C, t)(*(register + [ancilla])),
        pe(*(register + [memory]))**-1,
        cirq.measure(ancilla, key='a')
    ])
    c.append([
        cirq.PhasedXPowGate(
            exponent=sympy.Symbol('exponent'),
            phase_exponent=sympy.Symbol('phase_exponent'))(memory),
        cirq.measure(memory, key='m')
    ])
    return c
def simulate(circuit):
    simulator = cirq.Simulator()
    # Cases for measuring X, Y, and Z (respectively) on the memory qubit.
    params = [{
        'exponent': 0.5,
        'phase_exponent': -0.5
    }, {
        'exponent': 0.5,
        'phase_exponent': 0
    }, {
        'exponent': 0,
        'phase_exponent': 0
    }]
    print(circuit)
    results = simulator.run_sweep(circuit, params, repetitions=5000)
    for label, result in zip(('X', 'Y', 'Z'), list(results)):
        # Only select cases where the ancilla is 1.
        # TODO optimize using amplitude amplification algorithm
        expectation = 1 - 2 * np.mean(
            result.measurements['m'][result.measurements['a'] == 1])
        print('{} = {}'.format(label, expectation))
def main_cirq():
    """
    Simulates HHL with matrix input, and outputs Pauli observables of the
    resulting qubit state |x>.
    Expected observables are calculated from the expected solution |x>.
    """
    # Eigendecomposition:
    # >>> import numpy as np
    # >>> x, y = np.linalg.eigh(A) # Eigendecomposition for a complex Hermitian 
    # matrix.
    # >>> [z for z in zip(list(x.astype(np.float32)), list(np.transpose(y)))]
    # [(0.34899944, array([-0.23681357+0.j,  0.23727026-0.94213702j])),
    #  (4.5370007,  array([-0.97155511+0.j, -0.05783389+0.229643j]))]
    # |b> = (0.64510-0.47848j, 0.35490-0.47848j)
    # |x> = (-0.0662724-0.214548j, 0.784392-0.578192j)
    A = np.array([[4.30213466-6.01593490e-08j,
                   0.23531802+9.34386156e-01j],
                  [0.23531882-9.34388383e-01j,
                   0.58386534+6.01593489e-08j]])
    t = 0.358166*math.pi
    register_size = 4
    input_prep_gates = [cirq.rx(1.276359), cirq.rz(1.276359)]
    expected = (0.144130, 0.413217, -0.899154)
    # Set C to be the smallest eigenvalue that can be represented by the
    # circuit.
    C = 2*math.pi / (2**register_size * t)
    # Simulate circuit
    print("Expected observable outputs:")
    print("X =", expected[0])
    print("Y =", expected[1])
    print("Z =", expected[2])
    print("Actual: ")
    simulate(hhl_circuit(A, C, t, register_size, *input_prep_gates))

In [79]:
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [31]:
main_cirq()

Expected observable outputs:
X = 0.14413
Y = 0.413217
Z = -0.899154
Actual: 
0: ───────────────────────────────────────────────────────────────────────────────────#5──────────────────────────────────────────────────M('a')──────────────────────────────────────────────────────────────────────────────────────────────
                                                                                      │
1: ─────────────────────────────<__main__.PhaseEstimation object at 0x7fba30543070>───<__main__.EigenRotation object at 0x7fba305436d0>───(<__main__.PhaseEstimation object at 0x7fba30543070>**-1)───────────────────────────────────────────
                                │                                                     │                                                   │
2: ─────────────────────────────#2────────────────────────────────────────────────────#2──────────────────────────────────────────────────#2─────────────────────────────────────────────────────────────────────────────

In [6]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import numpy as np

shots = 8192

qc = QuantumCircuit()

q = QuantumRegister(3, 'q')
c = ClassicalRegister(3, 'c')

qc.add_register(q)
qc.add_register(c)
qc.ry(2.7005968494387864, q[2])
qc.measure(q[0], c[0])
qc.measure(q[1], c[1])
qc.measure(q[2], c[2])

backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend=backend, shots=shots)
job_result = job.result()
print(job_result.get_counts(qc))

{'000': 423, '100': 7769}


In [18]:
import qiskit
from qiskit.circuit.library.standard_gates import XGate

In [23]:
controlled_qubits = range(2)
a = qiskit.circuit.ControlledGate(
        'trial_gate', len(controlled_qubits)+1, [],
        num_ctrl_qubits=len(controlled_qubits),
        ctrl_state='0'*len(controlled_qubits),
        base_gate=XGate())
circ = QuantumCircuit(10)
circ.append(a)
circ.draw()

CircuitError: 'The amount of qubit(0)/clbit(0) arguments does not match the gate expectation (3).'

In [76]:
class PhaseEstimation_Qiskit(cirq.Gate):   
    # TODO: fix the subclass with something existing in Qiskit
    """
    A gate for Quantum Phase Estimation.
    unitary is the unitary gate whose phases will be estimated.
    The last qubit stores the eigenvector; all other qubits store the
    estimated phase, in big-endian.
    """
    def __init__(self, num_qubits, unitary):
        self._num_qubits = num_qubits
        self.U = unitary
    def num_qubits(self):
        return self._num_qubits
    def _decompose_(self, qubits):
        qubits = list(qubits)
        # yield an Hadamard gate to all the qubits except the last one
        # yield cirq.H.on_each(*qubits[:-1])
        # TODO
        
        # yiel the phase kick back (this gate itself) but acting on 
        # all the qubits this time
        # yield PhaseKickback(self.num_qubits(), self.U)(*qubits)
        # TODO recursive + implicit call of gate(something)
        
        # yield the QFT acting on all the qubits exept the last one
        # yield cirq.qft(*qubits[:-1], without_reverse=True)**-1
        # TODO QFT
            
class HamiltonianSimulation_Qiskit(cirq.EigenGate, cirq.SingleQubitGate):   
    # TODO: fix the subclass with something existing in Qiskit
    """
    A gate that represents e^iAt.
    This EigenGate + np.linalg.eigh() implementation is used here
    purely for demonstrative purposes. If a large matrix is used,
    the circuit should implement actual Hamiltonian simulation,
    by using the linear operators framework in Cirq for example.
    """
    def __init__(self, A, t, exponent=1.0):
        
        # this gate has two parent classes so we need to trigger both
        # - A gate that must be applied to exactly one qubit (SingleQubitGate).
        # - A gate with a known eigendecomposition (EigenGate).
        
        # cirq.SingleQubitGate.__init__(self)
        # cirq.EigenGate.__init__(self, exponent=exponent)
        
        # TODO
        # TODO
        
        self.A = A  # a MATRIX
        self.t = t  # algorithm parameter
        ws, vs = np.linalg.eigh(A)
        self.eigen_components = []
        for w, v in zip(ws, vs.T):
            theta = w*t / math.pi
            P = np.outer(v, np.conj(v))
            self.eigen_components.append((theta, P))
    def _with_exponent(self, exponent):
        # recursive call to perform the exponentiation multiple times
        return HamiltonianSimulation_Qiskit(self.A, self.t, exponent)
    def _eigen_components(self):
        return self.eigen_components

class PhaseKickback_Qiskit(cirq.Gate):  
    # TODO: fix the subclass with something existing in Qiskit
    """
    A gate for the phase kickback stage of Quantum Phase Estimation.
    It consists of a series of controlled e^iAt gates with the memory qubit as
    the target and each register qubit as the control, raised
    to the power of 2 based on the qubit index.
    unitary is the unitary gate whose phases will be estimated.
    """
    def __init__(self, num_qubits, unitary):
        # call the initializer of the Gate class
        # super(PhaseKickback, self)
        # TODO
        self._num_qubits = num_qubits
        self.U = unitary
    def num_qubits(self):
        return self._num_qubits
    def _decompose_(self, qubits):
        qubits = list(qubits)
        memory = qubits.pop()
        for i, qubit in enumerate(qubits):
            # create a controlled gate which acts on the 
            # exponentiate unitary gate and has 
            # one qubit and one [fixed] memory as control qubits
            
            # yield cirq.ControlledGate(self.U**(2**i))(qubit, memory)
            
            # TODO
class EigenRotation_Qiskit(cirq.Gate):
    # TODO: fix the subclass with something existing in Qiskit
    """
    EigenRotation performs the set of rotation on the ancilla qubit equivalent
    to division on the memory register by each eigenvalue of the matrix. The
    last qubit is the ancilla qubit; all remaining qubits are the register,
    assumed to be big-endian.
    It consists of a controlled ancilla qubit rotation for each possible value
    that can be represented by the register. Each rotation is a Ry gate where
    the angle is calculated from the eigenvalue corresponding to the register
    value, up to a normalization factor C.
    """
    def __init__(self, num_qubits, C, t):
        # call the initializer of the Gate class
        # super(EigenRotation, self)
        # TODO
        self._num_qubits = num_qubits
        self.C = C
        self.t = t
        self.N = 2**(num_qubits-1)
    def num_qubits(self):
        return self._num_qubits
    def _decompose_(self, qubits):
        for k in range(self.N):
            kGate = self._ancilla_rotation(k)
            # xor's 1 bits correspond to X gate positions.
            xor = k ^ (k-1)
            for q in qubits[-2::-1]:
                # Place X gates
                if xor % 2 == 1:
                    # yield a NOT gate on qubit q
                    # yield cirq.X(q)
                    # TODO
                xor >>= 1
                # Build controlled ancilla rotation
                # kGate = cirq.ControlledGate(kGate)
                # TODO
            yield kGate(*qubits)
    def _ancilla_rotation(self, k):
        if k == 0:
            k = self.N
        theta = 2*math.asin(self.C * self.N * self.t / (2*math.pi * k))
        # return a rotation y gate
        # return cirq.ry(theta)
        # TODO
def hhl_circuit(A, C, t, register_size, *input_prep_gates):
    """
    Constructs the HHL circuit.
    Args:
        A: The input Hermitian matrix.
        C: Algorithm parameter, see above.
        t: Algorithm parameter, see above.
        register_size: The size of the eigenvalue register.
        memory_basis: The basis to measure the memory in, one of 'x', 'y', 'z'.
        input_prep_gates: A list of gates to be applied to |0> to generate the
            desired input state |b>.
    Returns:
        The HHL circuit. The ancilla measurement has key 'a' and the memory
        measurement is in key 'm'.  There are two parameters in the circuit,
        `exponent` and `phase_exponent` corresponding to a possible rotation
        applied before the measurement on the memory with a
        `cirq.PhasedXPowGate`.
    """
    # prepare one quantum bit for the ancilla
    # ancilla = cirq.LineQubit(0)
    # TODO
    
    # prepare a quantum register of size: register_size 
    # to store eigenvalues of the matrix
    # register = [cirq.LineQubit(i + 1) for i in range(register_size)]
    # TODO
    
    # prepare an extra quantum bit for the input and output
    # to store input and output vectors
    # memory = cirq.LineQubit(register_size + 1)
    # TODO
    
    # create a new empty circuit
    # c = cirq.Circuit()
    # TODO
    
    hs = HamiltonianSimulation_Qiskit(A, t)
    
    # Quantum Phase Estimation Algo accoridng to Qiskit doc
    # Given a unitary operator U , the algorithm estimates θ in 
    # U | ψ ⟩ = e**(2*π*i*θ) | ψ ⟩ . 
    # Here | ψ ⟩ is an eigenvector and e**(2*π*i*θ) is the corresponding 
    # eigenvalue. Since U is unitary, all of its eigenvalues have a norm of 1.
    # Note that this algorithm contains a QFT inside     
    pe = PhaseEstimation_Qiskit(register_size+1, hs)
    
    c.append([gate(memory) for gate in input_prep_gates])
    c.append([
        pe(*(register + [memory])),
        EigenRotation(register_size + 1, C, t)(*(register + [ancilla])),
        pe(*(register + [memory]))**-1,
        cirq.measure(ancilla, key='a')
    ])
    c.append([
        cirq.PhasedXPowGate(
            exponent=sympy.Symbol('exponent'),
            phase_exponent=sympy.Symbol('phase_exponent'))(memory),
        cirq.measure(memory, key='m')
    ])
    return c
def simulate(circuit):
    simulator = cirq.Simulator()
    # Cases for measuring X, Y, and Z (respectively) on the memory qubit.
    params = [{
        'exponent': 0.5,
        'phase_exponent': -0.5
    }, {
        'exponent': 0.5,
        'phase_exponent': 0
    }, {
        'exponent': 0,
        'phase_exponent': 0
    }]
    results = simulator.run_sweep(circuit, params, repetitions=5000)
    for label, result in zip(('X', 'Y', 'Z'), list(results)):
        # Only select cases where the ancilla is 1.
        # TODO optimize using amplitude amplification algorithm
        expectation = 1 - 2 * np.mean(
            result.measurements['m'][result.measurements['a'] == 1])
        print('{} = {}'.format(label, expectation))
def main_qiskit():
    """
    Simulates HHL with matrix input, and outputs Pauli observables of the
    resulting qubit state |x>.
    Expected observables are calculated from the expected solution |x>.
    """
    # Eigendecomposition:
    # >>> import numpy as np
    # >>> x, y = np.linalg.eigh(A) # Eigendecomposition for a complex Hermitian 
    # matrix.
    # >>> [z for z in zip(list(x.astype(np.float32)), list(np.transpose(y)))]
    # [(0.34899944, array([-0.23681357+0.j,  0.23727026-0.94213702j])),
    #  (4.5370007,  array([-0.97155511+0.j, -0.05783389+0.229643j]))]
    # |b> = (0.64510-0.47848j, 0.35490-0.47848j)
    # |x> = (-0.0662724-0.214548j, 0.784392-0.578192j)
    A = np.array([[4.30213466-6.01593490e-08j,
                   0.23531802+9.34386156e-01j],
                  [0.23531882-9.34388383e-01j,
                   0.58386534+6.01593489e-08j]])
    t = 0.358166*math.pi
    register_size = 4
    input_prep_gates = [cirq.rx(1.276359), cirq.rz(1.276359)]
    expected = (0.144130, 0.413217, -0.899154)
    # Set C to be the smallest eigenvalue that can be represented by the
    # circuit.
    C = 2*math.pi / (2**register_size * t)
    # Simulate circuit
    print("Expected observable outputs:")
    print("X =", expected[0])
    print("Y =", expected[1])
    print("Z =", expected[2])
    print("Actual: ")
    simulate(hhl_circuit(A, C, t, register_size, *input_prep_gates))

IndentationError: expected an indented block (251332073.py, line 92)

## Alternative High Level API Call

In [27]:
np.array((0.144130, 0.413217, -0.899154)).dot(np.array((0.144130, 0.413217, -0.899154)).T)

0.999999661705

In [35]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.algorithms.linear_solvers.hhl import HHL
from qiskit.algorithms.linear_solvers.matrices import TridiagonalToeplitz
from qiskit.algorithms.linear_solvers.observables import MatrixFunctional

matrix = np.array([[4.30213466-6.01593490e-08j,
                   0.23531802+9.34386156e-01j],
                  [0.23531882-9.34388383e-01j,
                   0.58386534+6.01593489e-08j]])
right_hand_side = (0.144130, 0.413217, -0.899154) # already normalized
observable = MatrixFunctional(1, 1 / 2)
rhs = right_hand_side # already normalized

# Initial state circuit
num_qubits = 2
qc = QuantumCircuit(num_qubits)
qc.isometry(rhs, list(range(num_qubits)), None)

hhl = HHL()
solution = hhl.solve(matrix, qc, observable)
approx_result = solution.observable

QiskitError: 'The number of rows of the isometry is not a non negative power of 2.'

In [37]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.algorithms.linear_solvers.hhl import HHL
from qiskit.algorithms.linear_solvers.matrices import TridiagonalToeplitz
from qiskit.algorithms.linear_solvers.observables import MatrixFunctional

matrix = TridiagonalToeplitz(2, 1, 1 / 3, trotter_steps=2)
right_hand_side = [1.0, -2.1, 3.2, -4.3]
observable = MatrixFunctional(1, 1 / 2)
rhs = right_hand_side / np.linalg.norm(right_hand_side)

# Initial state circuit
num_qubits = matrix.num_state_qubits
qc = QuantumCircuit(num_qubits)
qc.isometry(rhs, list(range(num_qubits)), None)

hhl = HHL()
solution = hhl.solve(matrix, qc, observable)
approx_result = solution.observable

In [43]:
help(observable)

Help on MatrixFunctional in module qiskit.algorithms.linear_solvers.observables.matrix_functional object:

class MatrixFunctional(qiskit.algorithms.linear_solvers.observables.linear_system_observable.LinearSystemObservable)
 |  MatrixFunctional(main_diag: float, off_diag: int) -> None
 |  
 |  A class for the matrix functional of the vector solution to the linear systems.
 |  
 |  Examples:
 |  
 |      .. jupyter-execute::
 |  
 |          import numpy as np
 |          from qiskit import QuantumCircuit
 |          from qiskit.algorithms.linear_solvers.observables.matrix_functional import
 |           MatrixFunctional
 |          from qiskit.transpiler.passes import RemoveResetInZeroState
 |  
 |          tpass = RemoveResetInZeroState()
 |  
 |          vector = [1.0, -2.1, 3.2, -4.3]
 |          observable = MatrixFunctional(1, -1 / 3)
 |  
 |          init_state = vector / np.linalg.norm(vector)
 |          num_qubits = int(np.log2(len(vector)))
 |  
 |          # Get observable ci

In [38]:
qc.draw()

┌───────────┐
q_0: ┤0          ├
     │  Isometry │
q_1: ┤1          ├
     └───────────┘

## Operation Level

In [ ]:
"""
(0, 0): ─────────────────────────Ry(θ )─Ry(θ )─Ry(θ )─Ry(θ )──────────────M──₄₁₂₃
                     ┌──────┐    │      │      │      │ ┌───┐
(1, 0): ─H─@─────────│      │──X─@──────@────X─@──────@─│   │─────────@─H────
           │         │QFT^-1│    │      │      │      │ │QFT│         │
(2, 0): ─H─┼─────@───│      │──X─@────X─@────X─@────X─@─│   │─@───────┼─H────
           │     │   └──────┘                           └───┘ │       │
(3, 0): ───e^iAt─e^2iAt───────────────────────────────────────e^-2iAt─e^-iAt─

"""

In [ ]:
t = 0.358166*math.pi
A = np.array(
        [[4.30213466-6.01593490e-08j, 0.23531802+9.34386156e-01j],
        [0.23531882-9.34388383e-01j, 0.58386534+6.01593489e-08j]])

In [70]:
A.conjugate().T 

array([[4.30213466+6.01593490e-08j, 0.23531882+9.34388383e-01j],
       [0.23531802-9.34386156e-01j, 0.58386534-6.01593489e-08j]])

In [75]:
B = A * 1.j

def check_unitary(A):
    A_T_conj = A.transpose().conjugate() 
    return np.matmul(A_T_conj, A)

check_unitary(A)

array([[19.43681923-8.48990800e-18j,  1.1497642 +4.56541209e+00j],
       [ 1.1497642 -4.56541209e+00j,  1.26935079+6.50297966e-18j]])

In [63]:

check_unitary(A)

array([[18.50836263+0.j        , -0.81770481+0.43975707j],
       [-0.81770481-0.43975707j,  0.34089874+0.j        ]])

In [53]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import numpy as np

from qiskit.extensions import HamiltonianGate


shots = 8192

qc = QuantumCircuit()

q = QuantumRegister(4, 'q')
c = ClassicalRegister(4, 'c')

qc.add_register(q)
qc.add_register(c)
qc.h(q[1])
qc.h(q[2])

c_At_gate = HamiltonianGate(data=A, time=t).control(2)
qc.append(c_At_gate, q)

qc.measure(q[0], c[0])
qc.measure(q[1], c[1])
qc.measure(q[2], c[2])
qc.measure(q[3], c[3])
qc.barrier
qc.draw()

ExtensionError: 'Input matrix is not unitary.'

In [ ]:
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend=backend, shots=shots)
job_result = job.result()
print(job_result.get_counts(qc))

# Pytket Help

In [6]:
from pytket.extensions.qiskit import AerBackend
sim_b = AerBackend()
print(sim_b.required_predicates)

[NoSymbolsPredicate, GateSetPredicate:{ Unitary1qBox CnX ZZPhase Mølmer-Sørensen PhasedX Reset Rx H SXdg Tdg CSWAP T SWAP Sdg CCX S Y CU3 Unitary2qBox Z SX Measure RangePredicate X CU1 YYPhase Barrier U1 Ry Rz U3 U2 tk1 CX CY CZ noop }]


In [22]:
from pytket import Circuit
circ = Circuit(3, 3)
circ.Ry(0.7, 0)
circ.CX(0, 1)
circ.X(2)
circ.measure_all()

[Ry(0.7) q[0]; X q[2]; Measure q[2] --> c[2]; CX q[0], q[1]; Measure q[0] --> c[0]; Measure q[1] --> c[1]; ]

In [18]:
sim_b.compile_circuit(circ)
handle = sim_b.process_circuit(circ, n_shots=2000)
counts = sim_b.get_result(handle).get_counts()
print(counts)

Counter({(1, 1, 1): 1589, (0, 0, 1): 411})


In [27]:
from pytket.extensions.cirq import cirq_to_tk, tk_to_cirq

qc_cirq = tk_to_cirq(circ)
print(type(qc_cirq))
print(qc_cirq)

<class 'cirq.circuits.circuit.Circuit'>
0: ───Y^0.7───@───────────M('c[0]')───
              │
1: ───────────X───────────M('c[1]')───

2: ───X───────M('c[2]')───────────────


In [26]:
from pytket.extensions.qiskit import qiskit_to_tk, tk_to_qiskit

qc_qiskit = tk_to_qiskit(circ)
print(type(qc_qiskit))
print(qc_qiskit)

<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
     ┌───────────┐     ┌─┐   
q_0: ┤ Ry(7π/10) ├──■──┤M├───
     └───────────┘┌─┴─┐└╥┘┌─┐
q_1: ─────────────┤ X ├─╫─┤M├
         ┌───┐    └┬─┬┘ ║ └╥┘
q_2: ────┤ X ├─────┤M├──╫──╫─
         └───┘     └╥┘  ║  ║ 
c: 3/═══════════════╩═══╩══╩═
                    2   0  1 


In [25]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(circ)


In [32]:

A = np.array([[4.30213466-6.01593490e-08j,
               0.23531802+9.34386156e-01j],
              [0.23531882-9.34388383e-01j,
               0.58386534+6.01593489e-08j]])
t = 0.358166*math.pi
register_size = 4
input_prep_gates = [cirq.rx(1.276359), cirq.rz(1.276359)]
expected = (0.144130, 0.413217, -0.899154)
# Set C to be the smallest eigenvalue that can be represented by the
# circuit.
C = 2*math.pi / (2**register_size * t)
# Simulate circuit
print("Expected observable outputs:")
print("X =", expected[0])
print("Y =", expected[1])
print("Z =", expected[2])
print("Actual: ")
cirq_hll_circuit = hhl_circuit(A, C, t, register_size, *input_prep_gates)

Expected observable outputs:
X = 0.14413
Y = 0.413217
Z = -0.899154
Actual: 


In [33]:
print(cirq_hll_circuit)

0: ───────────────────────────────────────────────────────────────────────────────────#5──────────────────────────────────────────────────M('a')──────────────────────────────────────────────────────────────────────────────────────────────
                                                                                      │
1: ─────────────────────────────<__main__.PhaseEstimation object at 0x7fba3050bc40>───<__main__.EigenRotation object at 0x7fba3050bca0>───(<__main__.PhaseEstimation object at 0x7fba3050bc40>**-1)───────────────────────────────────────────
                                │                                                     │                                                   │
2: ─────────────────────────────#2────────────────────────────────────────────────────#2──────────────────────────────────────────────────#2──────────────────────────────────────────────────────────────────────────────────────────────────
                                │                      

In [34]:
qiskit_hll_circuit = tk_to_qiskit(cirq_to_tk(cirq_hll_circuit))

NotImplementedError: Operation not supported by tket: Rx(0.4062776880196569π)

In [37]:
from pytket.qasm import circuit_from_qasm, circuit_to_qasm_str
import tempfile, os

fd, path = tempfile.mkstemp(".qasm")
os.write(fd, """OPENQASM 2.0;
include "qelib1.inc";
qreg q[2];
creg c[2];
h q[0];
cx q[0], q[1];
cz q[1], q[0];
measure q -> c;
""".encode())
os.close(fd)
circ = circuit_from_qasm(path)
os.remove(path)

print(circuit_to_qasm_str(circ))
render_circuit_jupyter(circ)

OPENQASM 2.0;
include "qelib1.inc";

qreg q[2];
creg c[2];
h q[0];
cx q[0],q[1];
cz q[1],q[0];
measure q[0] -> c[0];
measure q[1] -> c[1];



## QASM Import

In [85]:
from functools import reduce
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import numpy as np
import re

def convert_to_both(circ):
    """Convert the circuit in both platforms."""
    qc_qiskit = tk_to_qiskit(circ)
    print(type(qc_qiskit))
    print(qc_qiskit)
    
    qc_cirq = my_tk_to_cirq(circ)
    print(type(qc_cirq))
    print(qc_cirq)
    
def execute_both(circ, shots=8192):
    
    
    # QISKIT
    qc_qiskit = tk_to_qiskit(circ)
    backend = Aer.get_backend('qasm_simulator')
    job = execute(qc_qiskit, backend=backend, shots=shots)
    job_result = job.result()
    counts_qiskit = job_result.get_counts(qc_qiskit)
    print(counts_qiskit)

    
    # CIRQ
    qc_cirq = my_tk_to_cirq(circ)
    simulator = cirq.Simulator()
    result = simulator.run(qc_cirq, repetitions=shots)
    measurement_keys = qc_cirq.all_measurement_key_names()
    register_numbers = [
        int(re.findall(r'\[(\d+)\]', measurement_key)[0])
        for measurement_key in measurement_keys
    ]
    sorted_measurement_keys = list(zip(*sorted(zip(register_numbers, measurement_keys))))[1]
    result_dict = dict(result.multi_measurement_histogram(keys=sorted_measurement_keys))
    keys = list(map(lambda arr: reduce(lambda x, y: str(x) + str(y), arr[::-1]), result_dict.keys()))
    counts_cirq = dict(zip(keys,[value for value in result_dict.values()]))
    print(counts_cirq)
    
    return counts_qiskit, counts_cirq

In [86]:
counts_qiskit, counts_cirq = execute_both(circ, shots=8192)

{'10000110000110000': 1, '00100110000110000': 1, '00000111010110000': 1, '10000111000110000': 1, '10001111000110000': 1, '00000001000000000': 1, '10101010000000010': 2, '00001111000110000': 3, '00101011000000010': 1, '10100000010000000': 5, '10100001000000010': 3, '10000110010110000': 2, '00001010010000010': 1, '10000100000110000': 1, '00101111000110011': 1, '00100100010110010': 1, '10101001000000010': 7, '10000011010000000': 2, '10001110000110000': 1, '00101110010110010': 3, '00000010000000000': 1, '00000100010110000': 1, '10000100010110000': 4, '00101110010110000': 2, '00100010010000010': 4, '00001001010000000': 1, '00000011000000000': 3, '00001101000110000': 1, '00100000010000000': 1, '00000110000110010': 2, '10101010000000000': 3, '10000101000110010': 1, '10001000000000000': 47, '00000101000110000': 1, '00100101000110010': 6, '10001010000000000': 29, '10100110000110010': 29, '10101011010000010': 677, '10101000000000010': 5, '00001111010110000': 17, '10101101010110000': 1, '00000001

In [87]:
import pandas as pd
import sys
sys.path.append("../lib")
import seaborn as sns
import matplotlib.pyplot as plt
from utils import iterate_over
from utils import load_config_and_check
from inspector import Inspector
from tqdm import tqdm
import os

In [88]:
%load_ext autoreload
%autoreload 2

In [89]:

from detectors import Faster_Energy_Detector
detector = Faster_Energy_Detector()
detector.check(counts_qiskit, counts_cirq)

n_1: 8192
a00: -1.4901161193847656e-08
a11: -1.4901161193847656e-08
a01: 1.4901161193847656e-08
d_1: tensor(1.3045e+08)
d_2: tensor(1.3024e+08)
d_12: tensor(1.3036e+08)
loss: tensor(0.0004)
<class 'torch.Tensor'>


(0.00044226646423339844, 1)

In [57]:
fd, path_qasm = tempfile.mkstemp(".qasm")
os.write(fd, """
OPENQASM 2.0;
include "qelib1.inc";

qreg q[17];
creg c[17];
rx(1.97669083856564*pi) q[0];
rx(0.7960885106608628*pi) q[1];
rz(0.35090351229498506*pi) q[3];
ry(0.4537029071284062*pi) q[4];
cx q[7],q[6];
rx(1.2688019171026421*pi) q[9];
rx(0.23123679015859144*pi) q[10];
u3(0.0*pi,0.9618638029687219*pi,0.0*pi) q[11];
rx(0.8462129202489219*pi) q[12];
cx q[15],q[16];
ry(1.7706536525502794*pi) q[1];
cx q[6],q[3];
cx q[4],q[11];
cx q[5],q[12];
rx(0.6245224459449306*pi) q[7];
cx q[8],q[16];
ry(1.2498070041911997*pi) q[9];
rx(0.2412573319806475*pi) q[10];
cx q[2],q[6];
rz(0.38844592115754173*pi) q[4];
rz(0.7235773112446283*pi) q[5];
cx q[7],q[14];
ry(0.3179192882894455*pi) q[9];
rz(1.0901360129329298*pi) q[10];
rz(1.2220470213551657*pi) q[12];
ry(0.7118297314349191*pi) q[16];
cx q[5],q[7];
rz(0.6335757942367595*pi) q[6];
ry(0.7779011482462892*pi) q[9];
rz(1.3834035974003542*pi) q[10];
ry(0.03225841339003366*pi) q[12];
ry(1.0208446749560223*pi) q[14];
cx q[11],q[5];
cx q[9],q[10];
ry(0.16638997666487754*pi) q[14];
cx q[5],q[7];
cx q[8],q[11];
u3(0.0*pi,0.7147951333663525*pi,0.0*pi) q[14];
ry(1.1065146895934268*pi) q[14];
cx q[2],q[14];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
measure q[4] -> c[4];
measure q[5] -> c[5];
measure q[6] -> c[6];
measure q[7] -> c[7];
measure q[8] -> c[8];
measure q[9] -> c[9];
measure q[10] -> c[10];
measure q[11] -> c[11];
measure q[12] -> c[12];
measure q[13] -> c[13];
measure q[14] -> c[14];
measure q[15] -> c[15];
measure q[16] -> c[16];""".encode())
os.close(fd)
circ = circuit_from_qasm(path_qasm)
os.remove(path_qasm)

print(circuit_to_qasm_str(circ))
render_circuit_jupyter(circ)

OPENQASM 2.0;
include "qelib1.inc";

qreg q[17];
creg c[17];
measure q[13] -> c[13];
rx(1.97669083856564*pi) q[0];
rx(0.7960885106608628*pi) q[1];
rz(0.35090351229498506*pi) q[3];
ry(0.4537029071284062*pi) q[4];
cx q[7],q[6];
rx(1.2688019171026421*pi) q[9];
rx(0.23123679015859144*pi) q[10];
u3(0.0*pi,0.9618638029687219*pi,0.0*pi) q[11];
rx(0.8462129202489219*pi) q[12];
cx q[15],q[16];
measure q[0] -> c[0];
measure q[15] -> c[15];
ry(1.7706536525502794*pi) q[1];
cx q[6],q[3];
cx q[4],q[11];
cx q[5],q[12];
rx(0.6245224459449306*pi) q[7];
cx q[8],q[16];
ry(1.2498070041911997*pi) q[9];
rx(0.2412573319806475*pi) q[10];
measure q[1] -> c[1];
measure q[3] -> c[3];
cx q[2],q[6];
rz(0.38844592115754173*pi) q[4];
rz(0.7235773112446283*pi) q[5];
cx q[7],q[14];
ry(0.3179192882894455*pi) q[9];
rz(1.0901360129329298*pi) q[10];
rz(1.2220470213551657*pi) q[12];
ry(0.7118297314349191*pi) q[16];
measure q[4] -> c[4];
measure q[16] -> c[16];
cx q[5],q[7];
rz(0.6335757942367595*pi) q[6];
ry(0.777901148246

In [81]:
import re
qc_cirq = my_tk_to_cirq(circ)
measurement_keys = qc_cirq.all_measurement_key_names()

register_numbers = [
    int(re.findall(r'\[(\d+)\]', measurement_key)[0])
    for measurement_key in measurement_keys
]
sorted_measurement_keys = list(zip(*sorted(zip(register_numbers, measurement_keys))))[1]
sorted_measurement_keys

('c[0]',
 'c[1]',
 'c[2]',
 'c[3]',
 'c[4]',
 'c[5]',
 'c[6]',
 'c[7]',
 'c[8]',
 'c[9]',
 'c[10]',
 'c[11]',
 'c[12]',
 'c[13]',
 'c[14]',
 'c[15]',
 'c[16]')

In [58]:
convert_to_both(circ)

<class 'qiskit.circuit.quantumcircuit.QuantumCircuit'>
         ┌──────────┐                 ┌─┐                              »
 q_0: ───┤ Rx(6.21) ├─────────────────┤M├──────────────────────────────»
        ┌┴──────────┤   ┌────────────┐└╥┘                        ┌─┐   »
 q_1: ──┤ Rx(2.501) ├───┤ Ry(5.5627) ├─╫─────────────────────────┤M├───»
        └───────────┘   └────────────┘ ║                         └╥┘   »
 q_2: ─────────────────────────────────╫──────────────────────■───╫────»
        ┌────────────┐      ┌───┐      ║                      │   ║ ┌─┐»
 q_3: ──┤ Rz(1.1024) ├──────┤ X ├──────╫──────────────────────┼───╫─┤M├»
        ├────────────┤      └─┬─┘      ║      ┌────────────┐  │   ║ └╥┘»
 q_4: ──┤ Ry(1.4253) ├────────┼────────╫───■──┤ Rz(1.2203) ├──┼───╫──╫─»
        └────────────┘        │        ║   │  └────────────┘  │   ║  ║ »
 q_5: ────────────────────────┼────────╫───┼────────■─────────┼───╫──╫─»
            ┌───┐             │        ║   │        │       ┌─┴─┐ ║  

In [50]:
from cirq.circuits.qasm_output import QasmUGate

In [55]:
# Copyright 2019-2022 Cambridge Quantum Computing
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""
Methods to allow conversion between Cirq and tket data types, including Circuits and
Devices
"""

from typing import List, Dict, FrozenSet, cast, Any
import cmath
from logging import warning
from cirq.devices import LineQubit, GridQubit
import cirq.ops
import cirq_google
from pytket.circuit import Circuit, OpType, Qubit, Bit, Node  # type: ignore
from pytket.routing import Architecture  # type: ignore
from sympy import pi  # type: ignore

# For translating cirq circuits to tket circuits
cirq_common = cirq.ops.common_gates
cirq_pauli = cirq.ops.pauli_gates
from cirq.circuits.qasm_output import QasmUGate

cirq_CH = cirq_common.H.controlled(1)

# map cirq common gates to pytket gates
_cirq2ops_mapping = {
    cirq_common.CNOT: OpType.CX,
    cirq_common.H: OpType.H,
    cirq_common.MeasurementGate: OpType.Measure,
    cirq_common.XPowGate: OpType.Rx,
    cirq_common.YPowGate: OpType.Ry,
    cirq_common.ZPowGate: OpType.Rz,
    cirq_common.XPowGate(exponent=0.5): OpType.V,
    cirq_common.XPowGate(exponent=-0.5): OpType.Vdg,
    cirq_common.S: OpType.S,
    cirq_common.SWAP: OpType.SWAP,
    cirq_common.T: OpType.T,
    cirq_pauli.X: OpType.X,
    cirq_pauli.Y: OpType.Y,
    cirq_pauli.Z: OpType.Z,
    cirq.ops.I: OpType.noop,
    cirq_common.CZPowGate: OpType.CU1,
    cirq_common.CZ: OpType.CZ,
    cirq_CH: OpType.CH,
    cirq.ops.CSwapGate: OpType.CSWAP,
    cirq_common.ISwapPowGate: OpType.ISWAP,
    cirq_common.ISWAP: OpType.ISWAPMax,
    cirq.ops.FSimGate: OpType.FSim,
    cirq_google.SYC: OpType.Sycamore,
    cirq.ops.parity_gates.ZZPowGate: OpType.ZZPhase,
    cirq.ops.parity_gates.XXPowGate: OpType.XXPhase,
    cirq.ops.parity_gates.YYPowGate: OpType.YYPhase,
    cirq.ops.PhasedXPowGate: OpType.PhasedX,
    cirq.ops.PhasedISwapPowGate: OpType.PhasedISWAP,
    QasmUGate: OpType.U3
}
# reverse mapping for convenience
_ops2cirq_mapping: Dict = dict((item[1], item[0]) for item in _cirq2ops_mapping.items())
# spot special rotation gates
_constant_gates = (
    cirq_common.CNOT,
    cirq_common.H,
    cirq_common.S,
    cirq_common.SWAP,
    cirq_common.T,
    cirq_pauli.X,
    cirq_pauli.Y,
    cirq_pauli.Z,
    cirq_common.CZ,
    cirq_CH,
    cirq_common.ISWAP,
    cirq_google.SYC,
    cirq.ops.I,
)
_rotation_types = (
    cirq_common.XPowGate,
    cirq_common.YPowGate,
    cirq_common.ZPowGate,
    cirq_common.CZPowGate,
    cirq_common.ISwapPowGate,
    cirq.ops.parity_gates.ZZPowGate,
    cirq.ops.parity_gates.XXPowGate,
    cirq.ops.parity_gates.YYPowGate,
)


def cirq_to_tk(circuit: cirq.circuits.Circuit) -> Circuit:
    """Converts a Cirq :py:class:`Circuit` to a tket :py:class:`Circuit` object.

    :param circuit: The input Cirq :py:class:`Circuit`

    :raises NotImplementedError: If the input contains a Cirq :py:class:`Circuit`
        operation which is not yet supported by pytket

    :return: The tket :py:class:`Circuit` corresponding to the input circuit
    """
    tkcirc = Circuit()
    qmap = {}
    for qb in circuit.all_qubits():
        if isinstance(qb, LineQubit):
            uid = Qubit("q", qb.x)
        elif isinstance(qb, GridQubit):
            uid = Qubit("g", qb.row, qb.col)
        elif isinstance(qb, cirq.ops.NamedQubit):
            uid = Qubit(qb.name)
        else:
            raise NotImplementedError("Cannot convert qubits of type " + str(type(qb)))
        tkcirc.add_qubit(uid)
        qmap.update({qb: uid})
    for moment in circuit:
        for op in moment.operations:
            if isinstance(op, cirq.ops.GlobalPhaseOperation):
                tkcirc.add_phase(cmath.phase(op.coefficient) / pi)
                continue
            gate = op.gate
            gatetype = type(gate)
            qb_lst = [qmap[q] for q in op.qubits]

            if isinstance(gate, cirq_common.HPowGate) and gate.exponent == 1:
                gate = cirq_common.H
            elif (
                gatetype == cirq_common.CNotPowGate
                and cast(cirq_common.CNotPowGate, gate).exponent == 1
            ):
                gate = cirq_common.CNOT
            elif (
                gatetype == cirq_pauli._PauliX
                and cast(cirq_pauli._PauliX, gate).exponent == 1
            ):
                gate = cirq_pauli.X
            elif (
                gatetype == cirq_pauli._PauliY
                and cast(cirq_pauli._PauliY, gate).exponent == 1
            ):
                gate = cirq_pauli.Y
            elif (
                gatetype == cirq_pauli._PauliZ
                and cast(cirq_pauli._PauliZ, gate).exponent == 1
            ):
                gate = cirq_pauli.Z

            apply_in_parallel = False
            if isinstance(gate, cirq.ops.ParallelGate):
                if gate.num_copies != len(qb_lst):
                    raise NotImplementedError(
                        "ParallelGate parameters defined incorrectly."
                    )
                gate = gate.sub_gate
                gatetype = type(gate)
                apply_in_parallel = True

            if gate in _constant_gates:
                try:
                    optype = _cirq2ops_mapping[gate]
                except KeyError as error:
                    raise NotImplementedError(
                        "Operation not supported by tket: " + str(op.gate)
                    ) from error
                params = []
            elif isinstance(gate, cirq_common.MeasurementGate):
                uid = Bit(gate.key)
                tkcirc.add_bit(uid)
                tkcirc.Measure(*qb_lst, uid)
                continue
            elif isinstance(gate, cirq.ops.PhasedXPowGate):
                optype = OpType.PhasedX
                pe = gate.phase_exponent
                params = [gate.exponent, pe]
            elif isinstance(gate, cirq.ops.FSimGate):
                optype = OpType.FSim
                params = [gate.theta / pi, gate.phi / pi]
            elif isinstance(gate, cirq.ops.PhasedISwapPowGate):
                optype = OpType.PhasedISWAP
                params = [gate.phase_exponent, gate.exponent]
            else:
                try:
                    optype = _cirq2ops_mapping[gatetype]
                    params = [cast(Any, gate).exponent]
                except (KeyError, AttributeError) as error:
                    raise NotImplementedError(
                        "Operation not supported by tket: " + str(op.gate)
                    ) from error
            if apply_in_parallel:
                for qb in qb_lst:
                    tkcirc.add_gate(optype, params, [qb])
            else:
                tkcirc.add_gate(optype, params, qb_lst)
    return tkcirc


def my_tk_to_cirq(tkcirc: Circuit, copy_all_qubits: bool = False) -> cirq.circuits.Circuit:
    """Converts a tket :py:class:`Circuit` object to a Cirq :py:class:`Circuit`.

    :param tkcirc: The input tket :py:class:`Circuit`

    :return: The Cirq :py:class:`Circuit` corresponding to the input circuit
    """
    if copy_all_qubits:
        tkcirc = tkcirc.copy()
        for q in tkcirc.qubits:
            tkcirc.add_gate(OpType.noop, [q])

    qmap = {}
    line_name = None
    grid_name = None
    # Since Cirq can only support registers of up to 2 dimensions, we explicitly
    # check for 3-dimensional registers whose third dimension is trivial.
    # SquareGrid architectures are of this form.
    indices = [qb.index for qb in tkcirc.qubits]
    is_flat_3d = all(idx[2] == 0 for idx in indices if len(idx) == 3)
    for qb in tkcirc.qubits:
        if len(qb.index) == 0:
            qmap.update({qb: cirq.ops.NamedQubit(qb.reg_name)})
        elif len(qb.index) == 1:
            if line_name != None and line_name != qb.reg_name:
                raise NotImplementedError(
                    "Cirq can only support a single linear register"
                )
            line_name = qb.reg_name
            qmap.update({qb: LineQubit(qb.index[0])})
        elif len(qb.index) == 2 or (len(qb.index) == 3 and is_flat_3d):
            if grid_name != None and grid_name != qb.reg_name:
                raise NotImplementedError(
                    "Cirq can only support a single grid register"
                )
            grid_name = qb.reg_name
            qmap.update({qb: GridQubit(qb.index[0], qb.index[1])})
        else:
            raise NotImplementedError(
                "Cirq can only support registers of dimension <=2"
            )
    oplst = []
    for command in tkcirc:
        op = command.op
        optype = op.type
        try:
            gatetype = _ops2cirq_mapping[optype]
        except KeyError as error:
            raise NotImplementedError(
                "Cannot convert tket Op to Cirq gate: " + op.get_name()
            ) from error
        if optype == OpType.Measure:
            qid = qmap[command.args[0]]
            bit = command.args[1]
            cirqop = cirq.ops.measure(qid, key=bit.__repr__())
        else:
            qids = [qmap[qbit] for qbit in command.args]
            params = op.params
            if len(params) == 0:
                cirqop = gatetype(*qids)
            elif optype == OpType.PhasedX:
                cirqop = gatetype(phase_exponent=params[1], exponent=params[0])(*qids)
            elif optype == OpType.FSim:
                cirqop = gatetype(
                    theta=float(params[0] * pi), phi=float(params[1] * pi)
                )(*qids)
            elif optype == OpType.PhasedISWAP:
                cirqop = gatetype(phase_exponent=params[0], exponent=params[1])(*qids)
            elif optype == OpType.U3:
                cirqop = gatetype(theta=params[0], phi=params[1], lmda=params[2])(*qids)
            else:
                cirqop = gatetype(exponent=params[0])(*qids)
        oplst.append(cirqop)
    try:

        coeff = cmath.exp(float(tkcirc.phase) * cmath.pi * 1j)
        if coeff.real < 1e-8:  # tolerance permitted by cirq for GlobalPhaseOperation
            coeff = coeff.imag * 1j
        if coeff.imag < 1e-8:
            coeff = coeff.real
        if coeff != 1.0:
            oplst.append(cirq.ops.GlobalPhaseOperation(coeff))
    except ValueError:
        warning(
            "Global phase is dependent on a symbolic parameter, so cannot adjust for "
            "phase"
        )
    return cirq.circuits.Circuit(*oplst)


# For converting cirq devices to tket devices


def _sort_row_col(qubits: FrozenSet[GridQubit]) -> List[GridQubit]:
    """Sort grid qubits first by row then by column"""

    return sorted(qubits, key=lambda x: (x.row, x.col))


def process_characterisation(xmon: cirq_google.XmonDevice) -> dict:
    """Generates a tket dictionary containing device characteristics for a Cirq
    :py:class:`XmonDevice`.

    :param xmon: The device to convert

    :return: A dictionary containing device characteristics
    """
    qb_map = {q: Node("q", q.row, q.col) for q in xmon.qubits}

    indexed_qubits = _sort_row_col(xmon.qubits)
    coupling_map = []
    for qb in indexed_qubits:
        neighbours = xmon.neighbors_of(qb)
        for x in neighbours:
            coupling_map.append((qb_map[qb], qb_map[x]))
    arc = Architecture(coupling_map)

    characterisation = dict()
    characterisation["Architecture"] = arc

    return characterisation
